## Creando una Vista hardcodeada

Views - vistas
- Vistas basadas en funciones
- Vistas basadas en clases

### Cómo crear una aplicación en Django

python manage.py startapp <nombre_de_la_app>

python manage.py startapp ecommerce

- funciones - nombrarlas con minúsculas
- clases - nombrarlas comenzando con mayúsculas

### ecommerce / views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse

# Create your views here.
def home(request):
    return HttpResponse("<h1>Hola Mundo</h1>")

### ecommerce / urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.home, name="home"),
]

### config / urls.py

In [ ]:
from django.contrib import admin
from django.urls import include
from django.urls import path

urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("ecommerce/", include("ecommerce.urls")), #<-------------------------- Lo que se agregó Django views
    path("admin/", admin.site.urls),
    path("__debug__/", include("debug_toolbar.urls")),
]

## Respuesta http y redireccionamiento

middlewares: procesos intermedios entre un request y un response

In [ ]:
# Create your views here.
def home(request):
    html = """
    <!DOCKTYPE html>
    <html>
        <head>
            <style>
                h1 {color: blue}
            </style>
        </head>

        <body>
            <h1>Hola Mundo</h1>
        </body>
    </html>
    """
    return HttpResponse(html)

# def home(request):
#     response = HttpResponse()
#     response.write("<p>Página no encontrada</p>")
#     response.status_code = 404
#     return response

def redirect_to_test(request):
    return HttpResponseRedirect("/ecommerce/")

### CRUD y Vistas

Forma dinámica de usar las vistas

CRUD - Create, Retrieve, Update, Delete
(Crear, Consultar, Actualizar, Eliminar)

- Crear modelos
- Agregar la app a INSTALLED_APPS
- Crear y aplicar migraciones
- Agregar admin

### models.py

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()

### config/settings.py

In [ ]:
...
# Application definitions
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig", #<------------------
    "debug_toolbar",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

...

## Cómo correr migraciones

- python manage.py makemigrations
- python manage.py migrate

## Cómo registrar modelos en el admin

In [ ]:
from django.contrib import admin

from .models import ProductModel

# Register your models here.
admin.site.register(ProductModel)

## Cómo crear un superuser para el admin
- python manage.py createsuperuser
- Password: samuel1998

## Tipos básicos de Vistas
- List View - donde ves los usuarios creados
- Create View - donde puedes insertar los datos para crear un usuario
- Retrieve and Update view - si le doy click an un usuario creado y lo modifico
- Delete view - Botón rojo para eliminar y confirmar

## ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

# Create your views here.
def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    return HttpResponse ("Un ecommerce personalizado")

## Usando Templates

### ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

# Create your views here.
def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    template = "ecommerce/list-view.html"
    context = {}
    return render(request, template, context)

### templates/ecommerce/list-view.html

In [ ]:
<h1>
Vista de Listado
</h1>

## Usando el contexto

### ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

# Create your views here.
def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    template = "ecommerce/list-view.html"
    context = {
        "products": queryset #<---------
    }
    return render(request, template, context)

### templates/ecommerce/list-view.html

In [ ]:
<h1>
Vista de Listado
</h1>

{% for product in products %}
    <li>
        {{product.title}} {{product.price}}
    </li>
{% endfor %}

## Protegiendo los endpoints

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

# Create your views here.
# @login_required
def product_model_list_view(request):
    print(request.user)
    queryset = ProductModel.objects.all()
    template = "ecommerce/list-view.html"
    context = {
        "products": queryset #<---------
    }
    
    if request.user.is_authenticated:
        template = "ecommerce/list-view.html"
    else:
        template = "ecommerce/list-view-public.html"

    return render(request, template, context)

@login_required
def login_required_view(request):
    print(request.user)
    queryset = ProductModel.objects.all()
    template = "ecommerce/list-view.html"
    context = {
        "products": queryset #<---------
    }
    
    if request.user.is_authenticated:
        template = "ecommerce/list-view.html"
    else:
        template = "ecommerce/list-view-public.html"

    return render(request, template, context)

### ecommerce/list-view-public.html

In [ ]:
<h1>
    Un ecommerce genial, regístrate
</h1>

## Vista de Detalle

### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

# Create your views here.


def product_model_detail_view(request, product_id):
    instance = get_object_or_404(ProductModel, id=product_id)
    context{
        "product":instance
    }
    template = "ecommerce/detail-view.html"
    return render(request, template, context)

# las demás views

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id", views.product_model_detail_view, name="detail"), #<-----------
]

### templates/ecommerce/detail-view.html

In [ ]:
<h1>
    {{ product.title }}
    {{ product.price }}
</h1>

### templates/ecommerce/list-view.html

In [ ]:
<h1>
Vista de Listado
</h1>
    
{% for product in products %}
<li>
    <a href="/ecommerce/{{ product.id }}"> {{ product.title }}</a>
</li>
{% endfor %}

## Vista de Creación

### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .forms import ProductModelForm
from .models import ProductModel

# Create your views here.

def product_model_create_view(request):
    form = ProductModelForm(request.POST or None)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        message.success(request, "Producto creado con éxito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
    context = {
        "form":form
    }
    template = "ecommerce/create-view.html"
    return render(request, template, context)

### templates/ecommerce/messages.html

In [ ]:
{% if messages %}
<ul class="messages">
    {% for message in messages %}
    <li{% if message.tags %} class="{{ message.tags }}"{% endif %}>{{ message }}</li>
    {% endfor %}
</ul>
{% endif %}

### templates/ecommerce/create-view.html

In [ ]:
{% include "ecommerce/messages.html" %}

<h1>
    Crear un nuevo producto
</h1>

<form method="POST" action="">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Crear">
</form>

Nota: Agregar

{% include "ecommerce/messages.html" %}

en todos los templates

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create", views.product_model_create_view, name="create"), #<----------
]

## Vista de Actualización

### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .forms import ProductModelForm
from .models import ProductModel

# Create your views here.

def product_model_update_view(request, product_id=None):
    instance = get_object_or_404(ProductModel, id=product_id)
    form = ProductModelForm(request.POST or None, instance=instance)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Producto actualizado con éxito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
    context = {
        "form":form
    }
    template = "ecommerce/update-view.html"
    return render(request, template, context)

### templates/ecommerce/update-view.html

In [ ]:
<h1>
Actualización de producto {{ form.instance.title }}
</h1>

{{ form.instance.title }}

<form method="POST" action="">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Actualizar">
</form>

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create", views.product_model_create_view, name="create"),
    path("<int:product_id>/edit/", views.product_model_update_view, name="update"),
]

## Vista para eliminar

### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .forms import ProductModelForm
from .models import ProductModel

# Create your views here.

def product_model_delete_view(request, product_id):
    instance = get_object_or_404(ProductModel, id=product_id)
    if request.method == "POST":
        instance.delete()
        HttpResponseRedirect("/ecommerce/")
        messages.success("Producto eliminado")
    context = {
        "product":instance
    }
    template = "ecommerce/delete-view.html"
    return render(request, template, context)

### templates/ecommerce/delete-view.html

In [ ]:
{% include "ecommerce/messages.html" %}

<h1>
Eliminar - {{ product.title }}
</h1>

<form method="POST" action="">
    {% csrf_token %}
    ¿Estás seguro que quieres eliminar el producto?
    <input type="submit" value="Eliminar">
    <a href="/ecommerce/{{ product.id }}">Cancelar</a>
</form>

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create", views.product_model_create_view, name="create"),
    path("<int:product_id>/edit/", views.product_model_update_view, name="update"),
    path("<int:product_id>/delete/", views.product_model_delete_view, name="delete"),
]

## Búsqueda en la Vista de Listado

### templates/ecommerce/search.html

In [ ]:
<form method="GET" action="/ecommerce/">
    <input type="text" name="q" placeholder="Buscar">
</form>

### templates/ecommerce/list-view.html

In [ ]:
{% include "ecommerce/search.html" %}

{% include "ecommerce/messages.html" %}

<h1>
Vista de Listado
</h1>
    
{% for product in products %}
<li>
    <a href="/ecommerce/{{ product.id }}"> {{ product.title }}</a>
</li>
{% endfor %}

### ecommmerce/views.py

In [ ]:
from django.db.models import Q

# @login_required
def product_model_list_view(request):
    query = request.GET.get("q",None)
    queryset = ProductModel.objects.all()
    if query is not None:
        queryset = queryset.filter(
            Q(title__icontains=query) |
            Q(price__icontains=query)
        )
    template = "ecommerce/list-view.html"
    context = {
        "products": queryset #<---------
    }
    
    if request.user.is_authenticated:
        template = "ecommerce/list-view.html"
    else:
        template = "ecommerce/list-view-public.html"

    return render(request, template, context)